** 7조 ML Poject 2 : 김태호, 서태민 <br>
참고로 BClassifier 모델은 LightGBM 모델을 설치하여 활용함

# Project 2 : Human Activity Recognition 

### 프로젝트 제출 가이드
- 제출 기한: 2023년 2월 5일(일)
- 제출 내용: `4. 구현` 셀에 5가지 분류기들을 구현하고, `Project2.ipynb`를 제출
- 제출 방법: `wisechang1@snu.ac.kr`로 이메일 제출 (여러 번 제출시 마지막 버전을 사용)
- 메일 제목: `X조 ML Project 2`
- 점수 비중: `Project 1 : Project 2 = 1 : 2`


### 채점 기준 [총 100점]
1. `LRClassifier`, `LDAClassifier`, `RFClassifier`, `BClassifier`, `SVMClassifier`, 각 분류기당 [+20점]
  1. 수행 [+15점] : `5. 테스트` 셀이 잘 수행되어야 하고, 각 분류기는 아래 주어진 방식으로 분류를 수행해야 합니다.
      - `LRClassifier` : Logistic Regression
      - `LDAClassifier` : Linear Discriminant Analysis
      - `RFClassifier` : Random Forest
      - `BClassifier` : Boosting with Decision Tree
      - `SVMClassifier` : Support Vector Machines
  2. 성능 [+5점]
    - 주어지지 않은 별개의 테스트 데이터를 이용해 각 분류기의 성능을 평가합니다. 각 분류기마다 팀별 상대적 성능을 고려해서 0점 ~ 5점 사이로 정해집니다. 팀별 성능을 점수로 변환하는 과정은 아래 코드를 참고해주세요.

    ```python
    import numpy as np
    from scipy import stats

    # Assume there are 5 teams. Example for single classifier.
    macro_f1_score_of_each_team = [0.9, 0.8, 0.5, 0.6, 0.65]

    # 1. Standardize f1 scores
    zscore = stats.zscore(macro_f1_score_of_each_team)
    # 2. Convert standardized f1 scores to 0 ~ 5 points
    score = 5 * (zscore - np.min(zscore)) / (np.max(zscore) - np.min(zscore))

    print(score)  # score = [5, 3.75, 0, 1.25, 1.875]
    ```

2. 띄어쓰기, 주석, Data Plotting 등 실행과 상관없는 부분은 수정이 가능하고, 실행과 관련된 부분은 `4. 구현` 셀만 수정하여야 합니다. [-50점]


### 참고 사항
1. `sklearn` 외의 다른 라이브러리를 사용해도 됩니다.
2. xyz축별 특성 뿐만 아니라 xyz축의 값을 합쳐서 사용해보셔도 좋습니다.
3. 어느 정도 성능이 나온 이후에는 Confusion Matrix를 그려보면서 헷갈리는 동작들을 구분할 수 있는 특성들을 추가하면 성능을 높일 수 있습니다.
4. 너무 많은 특성들을 사용하실 경우 Cross Validation 결과는 좋지만 Test 결과가 안좋을 수 있습니다.
5. 지금까지 각 분류기들의 최고 macro f1 score는 모두 0.75 정도 나왔습니다.

---
## 목차
1. 프로젝트 개요
2. 데이터<br/>
  2.1. 데이터 로드 방법<br/>
  2.2. 데이터 개요<br/>
  2.3. 데이터 샘플 예제<br/>
3. `KNNClassifier` 예제
4. 구현
5. 테스트

### 1. 프로젝트 개요
- 5초간 손목의 가속도계 센서 데이터를 이용해서 그 사람의 행동을 예측하는 분류기를 만듭니다.

<img src="https://gitlab.com/machine-learning-course1/ml-project-lg-2023-spring/-/raw/master/images/simple_pipeline.png" style='width:70%; margin-bottom:10px'>
<div style='text-align:center; font-weight:bold;'>그림 1. Walking, Running 예제 데이터</div>

### 2. 데이터
#### 2.1. 데이터 로드
- 아래와 같은 방법을 통해 `pandas.DataFrame` 포맷의 데이터를 얻을 수 있습니다.

In [ ]:
import pickle

BASE_DIR = '.'

with open(f'{BASE_DIR}/data/train.pkl', 'rb') as f:
    df = pickle.load(f)
df.head(3)

#### 2.2. 데이터 개요
- 각 샘플은 `Subject`, `Data`, `Motion` 값으로 이루어져 있습니다.
  - `Subject`: 실험자 ID (`int` 타입)
  - `Data`: (500, 3) 형태의 `numpy array` (`numpy array` 타입)
    - 5초 동안 100Hz로 3축 데이터를 모으면 (500, 3) 형태의 데이터가 됩니다.
  - `Motion`: 실험자의 실제 행동 ( `str` 타입)
    - ex. `walking`, `running`, ...

- `Data` 정보를 가지고 `Motion` 값을 유추해야합니다.
- `Subject`는 Cross Validation에 활용하기 위해 제공됩니다. Cross Validation의 경우 `CrossValidation.ipynb`를 참고해주세요.

#### 2.3. 데이터 샘플 예제

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


X, Y = np.stack(df['Data']), np.stack(df['Motion'])  # np.stack : pandas.Series => np.ndarray
groups = np.stack(df['Subject'])  # For LeaveOneGroupOut

X_sample, Y_sample = X[0], Y[0]  # First X - y pair
time_stamps = np.arange(0, 5, 1/100)                 # Time stamps for 100Hz * 5seconds

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(time_stamps, X_sample)
ax.set_title(Y_sample, fontsize=20)
ax.set_xlabel('Time (s)', fontsize=15)
ax.set_ylabel('Accelerometer ($m/s^2$)', fontsize=15)
fig.tight_layout()
fig.show()

### 3. `KNNClassifier` 예제
- 이 예제는 예시일 뿐이고, 다른 방식으로 구현하셔도 됩니다. 채점기준을 참고해주세요.

```python
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


def extract_features_from_a_single_sample(X_sample: np.ndarray) -> np.ndarray:
    """
    Extract features from a single sample

    Parameters
    ----------
    X_sample : array of shape (500, 3)
        100Hz * 5 seconds => 500
        3 axis (x, y, z)  => 3

    Returns
    -------
    features : array with (p,) shape
        Extracted features from X_sample
    """
    X_fft = np.abs(np.fft.fft(X_sample, axis=0))[1:]
    
    mean = np.mean(X_sample, axis=0)
    std = np.std(X_sample, axis=0)
    
    features = np.hstack([mean, std])
    return features


def extract_features_from_multiple_samples(X_samples: np.ndarray) -> np.ndarray:
    """
    Extract features from multiple samples

    Parameters
    ----------
    X_samples : array of shape (n, 500, 3)
        List of data samples

    Returns
    -------
    array of shape (n, p)
        Extracted features from X_samples
    """
    return np.asarray([extract_features_from_a_single_sample(X_sample) for X_sample in X_samples])


# Example Classifier
# You can use python class instance instead of the Pipeline.
KNNClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples)),
    ('preprocessor', PCA()),
    ('classifier', KNeighborsClassifier()),
])
```

#### `KNNClassifier` 구현 순서
##### a. `extract_features_from_a_single_sample`
- (500, 3) 크기의 데이터 샘플 하나를 입력받아 특성들을 1D numpy array로 반환합니다.
- 주어진 예제에서는 `mean`, `std` 특성들을 뽑아내고 있습니다.
- 추가적인 특성들을 구현할 때 아래 목록을 참고하실 수 있습니다.
  - Time Domain 특성
    - min, max, amplitude, sum, absolute sum, euclidean norm, mean, absolute mean, mean absolute deviation, variance, standard deviation, zero crossings, auto-correlations, skewness, kurtosis, percentiles, power, root mean square, cross-correlation of axes, ...
  - Frequency Domain 특성
    - energy, entropy, power, spectral centroid, mean, median, standard deviation, dominant frequency, ...

##### b. 전처리기 (Preprocessor) 추가
- a.에서 뽑아낸 특성들을 전처리합니다.
- 주어진 예제에는 `PCA`가 포함되어 있습니다.
- 아래 링크에서 `sklearn`이 제공하는 전처리기 목록을 확인하실 수 있습니다.
  - [`sklearn.preprocessing` documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing).
- `sklearn` 외의 라이브러리를 이용하셔도 됩니다.


##### c. 분류기 튜닝
- a.에서 어떤 특성을 사용할지, b.에서 어떤 전처리기를 사용할지, 전처리기/분류기의 하이퍼파라미터를 어떻게 정할지 등을 바꾸어가며 성능을 높입니다.
- Cross Validation을 이용해서 테스트 결과를 예측하며 하이퍼파라미터를 설정할 수 있습니다.
- 자세한 튜닝 방법은 `CrossValidation.ipynb`를 참고해주세요.

### 4. 구현

In [7]:
############################################################
# Implement your own classifiers in this cell

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from scipy.stats import skew, kurtosis, mode, entropy
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

def extract_features_from_a_single_sample(k, X_sample: np.ndarray) -> np.ndarray:
    """
    Extract features from a single sample

    Parameters
    ----------
    X_sample : array of shape (500, 3)
        100Hz * 5 seconds => 500
        3 axis (x, y, z)  => 3

    Returns
    -------
    features : array with (p,) shape
        Extracted features from X_sample
    """

    ###########################################################
    #################### Time Domain 특성 #####################
    ###########################################################
    time_min = np.min(X_sample, axis=0)
    time_max = np.max(X_sample, axis=0)
    
    time_euclidean_norm = np.sqrt(np.sum(X_sample*X_sample, axis=0))
    time_mean_absolute_deviation = np.mean(np.absolute(X_sample - np.mean(X_sample, axis=0)), axis=0)
    time_mean = np.mean(X_sample, axis=0)
    time_variance = np.var(X_sample, axis=0)
    time_std = np.std(X_sample, axis=0)
    time_zero_crossing = np.sum(np.diff(np.sign(X_sample), axis=0)!=0, axis=0)/len(X_sample)
    time_skewness = skew(X_sample, axis=0)
    time_kurtosis = kurtosis(X_sample, fisher=True)
    
    autocorr_x = np.corrcoef([i[0] for i in X_sample][1:], [i[0] for i in X_sample][:-1])[0,1]
    autocorr_y = np.corrcoef([i[1] for i in X_sample][1:], [i[1] for i in X_sample][:-1])[0,1]
    autocorr_z = np.corrcoef([i[2] for i in X_sample][1:], [i[2] for i in X_sample][:-1])[0,1]
    time_autocorr = np.array([autocorr_x, autocorr_y, autocorr_z])
    
    crosscorr_xy = np.corrcoef([i[0] for i in X_sample], [i[1] for i in X_sample])[0,1]
    crosscorr_xz = np.corrcoef([i[0] for i in X_sample], [i[2] for i in X_sample])[0,1]
    crosscorr_yz = np.corrcoef([i[1] for i in X_sample], [i[2] for i in X_sample])[0,1]
    time_crosscorr = np.array([crosscorr_xy, crosscorr_xz, crosscorr_yz])
    
    time_rms = np.sqrt(np.mean(X_sample**2, axis=0))
    time_peak_to_peak = np.max(X_sample,axis=0) - np.min(X_sample,axis=0)
    time_quan_1 = np.quantile(X_sample,axis=0, q=0.25)
    time_quan_3 = np.quantile(X_sample,axis=0, q=0.75) 
    time_iqr = time_quan_3-time_quan_1

    time_max_xy = time_max[0] - time_max[1]
    time_max_yz = time_max[1] - time_max[2]
    time_max_xz = time_max[2] - time_max[0]
    time_mean_xy = time_mean[0] - time_mean[1]
    time_mean_yz = time_mean[1] - time_mean[2]
    time_mean_xz = time_mean[2] - time_mean[0] 

    time_diff_xy = X_sample[:,0] - X_sample[:,1]
    time_diff_yz = X_sample[:,1] - X_sample[:,2]
    time_diff_xz = X_sample[:,2] - X_sample[:,0]
    
    time_diff_max_xy = np.max(time_diff_xy)
    time_diff_max_yz = np.max(time_diff_yz)
    time_diff_max_xz = np.max(time_diff_xz)
    time_diff_min_xy = np.min(time_diff_xy)
    time_diff_min_yz = np.min(time_diff_yz)
    time_diff_min_xz = np.min(time_diff_xy)
    
    time_xy_zero_crossing = np.sum(np.diff(np.sign(time_diff_xy), axis=0)!=0, axis=0)/len(time_diff_xy)
    time_yz_zero_crossing = np.sum(np.diff(np.sign(time_diff_yz), axis=0)!=0, axis=0)/len(time_diff_yz)
    time_xz_zero_crossing = np.sum(np.diff(np.sign(time_diff_xz), axis=0)!=0, axis=0)/len(time_diff_xz) 
    
    # x, y, z를 개별로 moving average max, min
    acc2_pd = pd.DataFrame(X_sample)
    accmax = acc2_pd.rolling(window=9).max()
    accmin = acc2_pd.rolling(window=9).min()
    accmaxmindiff = accmax.dropna() - accmin.dropna()    ## moving average max_min difference
    acc_stack = accmaxmindiff.to_numpy()
    acc_stack_mean = acc_stack.mean(axis = 0)
    acc_stack_std = acc_stack.std(axis = 0)
    acc_stack_max = acc_stack.max(axis = 0)
    acc_stack_norm = (acc_stack_max - acc_stack_mean) / acc_stack_std

    ###########################################################
    ################# Frequency Domain 특성 ###################
    ###########################################################

    X_fft = np.abs(np.fft.fft(X_sample, axis=0))[1:]
    X_fft_xyz = np.mean(X_fft, axis=1)
    f_front_mean_xyz = np.mean(X_fft_xyz[:4])
    f_middle_mean_xyz = np.mean(X_fft_xyz[4:29])
    f_back_mean_xyz = np.mean(X_fft_xyz[29:125])
    f_front_max_xyz = np.max(X_fft_xyz[:10])
    f_middle_max_xyz = np.max(X_fft_xyz[10:30])
    f_back_max_xyz = np.max(X_fft_xyz[30:125])
    
    freqs = np.fft.fftfreq(len(X_sample))[1:]
    
    f_energy = np.sum(X_fft**2, axis=0)
    f_max = np.max(X_fft, axis=0)    
    f_quan_1 = np.quantile(X_fft,axis=0, q=0.25)
    f_quan_3 = np.quantile(X_fft,axis=0, q=0.75)  
    f_iqr = f_quan_3-f_quan_1
    f_dominant_frequency = freqs[np.argmax(X_fft, axis=0)]    
    f_entropy = entropy(X_fft, axis=0)
    
    # Frequency 분석을 통해 각 Motion 특징을 구분할 수 있는 구간으로 나누어 해당 구간 Feature 추출
    f_front_energy = np.mean(X_fft[:4], axis=0)
    f_middle_energy = np.mean(X_fft[4:29], axis=0)
    f_back_energy = np.mean(X_fft[29:149], axis=0)
    f_ediff_frontmiddle = f_front_energy - f_middle_energy
    
    f_front_energy_xyz = np.sum(f_front_energy)
    f_middle_energy_xyz = np.sum(f_middle_energy)
    f_back_energy_xyz = np.sum(f_back_energy)
    
    f_front_max = np.max(X_fft[:4], axis=0)
    f_middle_max = np.max(X_fft[4:29], axis=0)
    f_back_max = np.max(X_fft[29:149], axis=0)
    f_mdiff_frontmiddle = f_front_max - f_middle_max

    f_front_energy_xy = f_front_energy[0] - f_front_energy[1]
    f_front_energy_yz = f_front_energy[1] - f_front_energy[2]
    f_front_energy_xz = f_front_energy[2] - f_front_energy[0]
    
    f_front_max_xy = f_front_max[0] - f_front_max[1]
    f_front_max_yz = f_front_max[1] - f_front_max[2]
    f_front_max_xz = f_front_max[2] - f_front_max[0]
    
    f_middle_energy_xy = f_middle_energy[0] - f_middle_energy[1]
    f_middle_energy_yz = f_middle_energy[1] - f_middle_energy[2]
    f_middle_energy_xz = f_middle_energy[2] - f_middle_energy[0]
    
    f_middle_max_xy = f_middle_max[0] - f_middle_max[1]
    f_middle_max_xz = f_middle_max[2] - f_middle_max[0]
 
    f_back_energy_xy = f_back_energy[0] - f_back_energy[1]
    f_back_energy_yz = f_back_energy[1] - f_back_energy[2]
    f_back_energy_xz = f_back_energy[2] - f_back_energy[0]
    
    f_back_max_xy = f_back_max[0] - f_back_max[1]
    f_back_max_xz = f_back_max[2] - f_back_max[0]

    # Feature 배열 결합
    features_other = np.hstack([time_min, time_max, time_euclidean_norm, time_mean, time_std, time_zero_crossing,\
                        time_skewness, time_kurtosis, time_autocorr, time_crosscorr, time_peak_to_peak,time_iqr,\
                        time_max_xy, time_max_yz, time_max_xz, time_mean_xy, time_mean_yz, time_mean_xz,\
                        f_energy, f_max, f_iqr, f_dominant_frequency, f_entropy,\
                        f_front_energy_xyz, f_middle_energy_xyz, f_back_energy_xyz,\
                        f_front_mean_xyz, f_middle_mean_xyz, f_back_mean_xyz, f_ediff_frontmiddle,\
                        f_front_max_xyz, f_middle_max_xyz, f_back_energy_xyz, f_mdiff_frontmiddle,\
                        f_front_energy_xy, f_front_energy_yz, f_front_energy_xz, f_front_max_xy, f_front_max_yz, f_front_max_xz,\
                        f_middle_energy_xy, f_middle_energy_xz, f_middle_max_xy, f_middle_max_xz,\
                        f_back_energy_xy, f_back_energy_yz , f_back_energy_xz, f_back_max_xy,f_back_max_xz,\
                        time_diff_max_xy, time_diff_max_yz, time_diff_max_xz,\
                        time_xy_zero_crossing, time_yz_zero_crossing, time_xz_zero_crossing, \
                        acc_stack_mean, acc_stack_norm]) 
    features_LR = np.hstack([time_euclidean_norm, time_mean_absolute_deviation, time_mean, time_variance, time_std, time_zero_crossing,\
                        time_skewness, time_kurtosis, time_autocorr, time_crosscorr, time_rms, time_peak_to_peak,\
                        f_front_mean_xyz, f_middle_mean_xyz, f_back_mean_xyz, f_front_max_xyz, f_middle_max_xyz, f_back_max_xyz,\
                        acc_stack_mean, acc_stack_std, acc_stack_max, acc_stack_norm])    

    if (k == 0):
        features = features_LR
    else:
        features = features_other
    
    return features
  
# sample for other classifiers (LinearDiscriminantAnalysis, RandomForestClassifier, LGBMClassifier)
def extract_features_from_multiple_samples(X_samples: np.ndarray) -> np.ndarray:
    """
    Extract features from multiple samples

    Parameters
    ----------
    X_samples : array of shape (n, 500, 3)
        List of data samples

    Returns
    -------
    array of shape (n, p)
        Extracted features from X_samples
    """
    return np.asarray([extract_features_from_a_single_sample(1, X_sample) for X_sample in X_samples])
# sample for LogisticRegression, SVC
def extract_features_from_multiple_samples_LR(X_samples: np.ndarray) -> np.ndarray:
    return np.asarray([extract_features_from_a_single_sample(0, X_sample) for X_sample in X_samples])

# Example Classifier
# You can use python class instance instead of the Pipeline.
LRClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples_LR)),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(C=1.3, solver='newton-cg')),
])
''''''
LDAClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples)),
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', LinearDiscriminantAnalysis(solver='svd')),
])

RFClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples)),
    ('preprocessor', StandardScaler()),
    ('classifier',RandomForestClassifier(n_estimators=700, random_state=5)),
])

BClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples)),
    ('preprocessor', StandardScaler()),
    ('classifier',LGBMClassifier(n_estimators=500, learning_rate=0.05)),
])

SVMClassifier = Pipeline([
    ('feature_extractor', FunctionTransformer(extract_features_from_multiple_samples_LR)),
    ('preprocessor', MinMaxScaler()),
    ('classifier',SVC(kernel='poly', C=8)),
])

############################################################

### 5. 테스트

In [8]:
# Import Libraries
import pickle
from sklearn.model_selection import cross_val_score, LeaveOneGroupOut

BASE_DIR = '.'

# Data Load
with open(f'{BASE_DIR}/data/train.pkl', 'rb') as f:
    df = pickle.load(f)

X, Y = np.stack(df['Data']), np.stack(df['Motion'])  # np.stack : pandas.Series => np.ndarray
groups = np.stack(df['Subject'])  # For LeaveOneGroupOut


# Scoring
for Classifier in [LRClassifier, LDAClassifier, RFClassifier, BClassifier, SVMClassifier]:
    scores = cross_val_score(
        Classifier,
        X, Y, groups=groups,
        scoring='f1_macro',
        cv=LeaveOneGroupOut())
    print(np.mean(scores))

0.6676460047380657
0.6454461997707358
0.6615384812165567
0.7403086924646182
0.6627523213353322
